## 3D Void Representative Cycles
Look at the 3D cycle representatives created via optimization.

#### Preliminaries

In [1]:
# load some packages
import plotly.graph_objects as go
import oatpy as oat
import numpy as np
import time # keep track of how long optmization takes

# enviorment
np.random.seed(10) # use numpy randomization to generate points
n = 150 # number of points

#### Data
We create a dataset that has two spheres connected by two bars.

In [2]:
def gen_sphere(n: int,
               r: float = 1,
               x: float = 0,
               y: float = 0,
               z: float = 0
               ) -> np.ndarray:
    '''
    Randomly create pointset along a radius r sphere with n points centered (x, y, z).
    '''
    theta = np.random.uniform(0, 2*np.pi, n) # use sphereical coords with random angles and set radius
    phi = np.random.uniform(0, 2*np.pi, n)

    X = r * np.sin(phi) * np.cos(theta) # convert to cartasian coords
    Y = r * np.sin(phi) * np.sin(theta)
    Z = r * np.cos(phi)

    pts = np.column_stack((X, Y, Z)) # combine, each row is one coord
    pts += np.array([x, y, z]) # center the circle at (x, y, z)
    return pts


def gen_bar(n: int,
            start: np.ndarray[float],
            end: np.ndarray[float],
            ) -> np.ndarray:
    '''
    Create a 3d pointset between the start and end points
    '''
    p = end-start # difference vector
    len = sum((p)**2)**(1/2) # disatnce from start to end
    R = np.random.uniform(0, len, n) # the locations of the points along the line from start to end
    theta = np.arctan2(p[1], p[0]) if [0] != 0 else np.sign(p[1]) * np.pi / 2 # spherical angles of the points
    phi = np.arctan2(sum((p[:2])**2)**(1/2), p[2])
    
    X = R * np.sin(phi) * np.cos(theta) # convert to cartasian coords
    Y = R * np.sin(phi) * np.sin(theta)
    Z = R * np.cos(phi)

    pts = np.column_stack((X, Y, Z)) # combine, each row is one coord
    pts += start # start the bar at the start
    return pts


def add_noise(pts: np.ndarray,
              sigma: float
              ) -> np.ndarray:
    '''
    Shifts the points around to add noise. Points move in a normally distrubuted ball around the current location
    '''
    n = pts.shape[0] # number of points
    R = np.random.normal(0, sigma, n) # distance from current point
    theta = np.random.uniform(0, 2*np.pi, n) # angles
    phi = np.random.uniform(0, 2*np.pi, n)

    X = R * np.sin(phi) * np.cos(theta) # noise from point in cartasian coords
    Y = R * np.sin(phi) * np.sin(theta)
    Z = R * np.cos(phi)

    noise = np.column_stack((X, Y, Z)) # combine, each row is one coord
    return pts + noise


def gen_points(n: int,
               r: float = 2,
               sigma: float = 0
               ) -> np.ndarray:
    '''
    Create an n pointed set around our shape with two spheres connected by tow bars.
    '''
    sphere_n = int(n // 2.3) # number of points in the sphere, 2.3 chosen based on vibes but it creates a nice shape imo
    bar_n = int((n - 2*sphere_n) / 2) # number of points in the bar

    top_sphere = gen_sphere(sphere_n, r, z=1.5*r) # gen the two spheres
    bottom_sphere = gen_sphere(sphere_n, r, z=-1.5*r)
    left_bar = gen_bar(bar_n, np.array([0, -r, -1.5*r]), np.array([0, -r, 1.5*r])) # gen the two bars
    right_bar = gen_bar(bar_n, np.array([0, r, -1.5*r]), np.array([0, r, 1.5*r]))

    pts = np.row_stack((top_sphere, bottom_sphere, left_bar, right_bar))
    pts = add_noise(pts, sigma=sigma) # add noise (I dont think this is nessisary but it feels better)
    return pts

In [3]:
dta = gen_points(n, sigma=0.1)

# plotly plotting
trace = go.Scatter3d(x=dta[:, 0], y=dta[:, 1], z=dta[:, 2], mode='markers')
fig = go.Figure(trace)
fig.update_layout(
        width=300, 
        height=500,
        margin=dict(l=20, r=20, t=20, b=20),
    )  
fig.show()

#### Solve Homology
We use `homology_dimension_max=2` here to find the trapped voids. It does take much longer tho.

In [4]:
# setup problem
enclosing = oat.dissimilarity.enclosing_from_cloud(dta) # max fintration radius
dissimilairty_matrix = oat.dissimilarity.matrix_from_cloud( # distance matrix
        cloud=dta,
        dissimilarity_max=enclosing + 1e-10 # i belive any elements past this are removed (returns a sparse matrix)
    )
# add 1e-10 to elimite some numerical error (greg says to do it)
factored = oat.rust.FactoredBoundaryMatrixVr( # two functions that do this, idk what the other one is
        dissimilarity_matrix=dissimilairty_matrix,
        homology_dimension_max=2
    )

# solve homology
homology = factored.homology( # solve homology
        return_cycle_representatives=True, # These need to be true to be able to make a barcode, makes the problem take ~30% longer (1:30ish)
        return_bounding_chains=False
    )

#### Visualize Homology
See a persistence diagram for the homology.

Based on the data input, we expect there to be only one 1D cycle, 1 large 2D cycle, and 2 large 3d cycles, which is what we see.

In [5]:
# Persistance diagram
fig = oat.plot.pd(homology)
fig.update_layout(
        width=600, 
        height=500,
        margin=dict(l=20, r=20, t=20, b=20)
    )
fig.show()

#### Find Representative 2D Cycles (Around 3D Voids)
To find representative cycles, we use the factored matrix and the `optimize_cycle` method.

We do this both the largest and second largest 3D Void.

In [6]:
## 2nd Largest Representative 3D Hole
# cycle to optimize
i = homology[homology['dimension'] == 2].sort_values('cycle nnz').index[-2] # 2nd largest 2d cycle (3d void)

# get the cycle
nonoptimal_faces = homology.loc[i, 'cycle representative']['simplex'].tolist()

# plotting
trace_dta = go.Scatter3d( # data plot
        x=dta[:, 0],
        y=dta[:, 1],
        z=dta[:, 2],
        mode='markers',
        showlegend=True,
        name='Data',
        opacity=0.5
    )
traces_nonoptimal = [oat.plot.triangle__trace3d(triangle=tri, coo=dta) for tri in nonoptimal_faces] # optimal cycle plot
for n, trace in enumerate(traces_nonoptimal): # plot optimal cycle
    trace.update(
            showlegend=(n==0),
            legendgroup='opti',
            opacity=0.5,
            name='Non-Optimal Cycle'
        )
fig = go.Figure(data=[trace_dta]+traces_nonoptimal)
fig.update_layout(
        width=400, 
        height=500,
        margin=dict(l=20, r=20, t=20, b=20)
    )
# fig.write_html('void_2.html')
fig.show()

In [7]:
## 2nd Largest Representative 3D Hole (Optimized)
# optimization problem
start = time.time()
optimal = factored.optimize_cycle(
        birth_simplex=homology['birth simplex'][i], # same i as above
        problem_type='preserve PH basis'
    )
print(f'Optimization took {time.time() - start} secs')
optimal_faces = optimal['chain']['optimal cycle']['simplex'].tolist() # bounding box of optimal cycle

# # plotting
# trace_dta = go.Scatter3d( # data plot
#         x=dta[:, 0],
#         y=dta[:, 1],
#         z=dta[:, 2],
#         mode='markers',
#         showlegend=True,
#         name='Data',
#         opacity=0.5
#     )
# traces_optimal = [oat.plot.triangle__trace3d(triangle=tri, coo=dta) for tri in optimal_faces] # optimal cycle plot
# for n, trace in enumerate(traces_optimal): # plot optimal cycle
#     trace.update(
#             showlegend=(n==0),
#             legendgroup='opti',
#             opacity=0.5,
#             name='Optimal Cycle'
#         )
# fig = go.Figure(data=[trace_dta]+traces_optimal)
# fig.update_layout(
#         width=400, 
#         height=500,
#         margin=dict(l=20, r=20, t=20, b=20)
#     )
# fig.show()
# # Essentially, there's a lot of degenerate faces that have a coefficeint close to 0 that aren't -1 or 1 in the optimization problem.
# # This causes the result to be an ugly mess that's not what we want.


Finished construcing L1 optimization program.
Constraint matrix has 65388 nonzero entries.
Passing program to solver.

Done solving.
MINILP solution: Solution { direction: Minimize, num_vars: 18138, num_constraints: 19828, objective: 60.65644241944831 }
Optimization took 73.00003123283386 secs


In [8]:
## 2nd Largest Representative 3D Hole (Optimized and rounded)
# use optimal dataframe from above (the optmization problem takes too long, i don't wanna do it twice)
filter = optimal.loc['optimal cycle', 'chain']['coefficient'].astype(float).round() != 0 # mark locations where the cofficicents are close to 0 to be removed
print(f'Removing {sum(1-filter)}/{len(filter)} simplicies from the cycle, {sum(filter)} simplicies left')
optimal_faces = optimal.loc['optimal cycle', 'chain'][filter]['simplex'].tolist()
# maybe want some assert statement to make sure we have a real cycle

# plotting
trace_dta = go.Scatter3d( # data plot
        x=dta[:, 0],
        y=dta[:, 1],
        z=dta[:, 2],
        mode='markers',
        showlegend=True,
        name='Data',
        opacity=0.5
    )
traces_optimal = [oat.plot.triangle__trace3d(triangle=tri, coo=dta) for tri in optimal_faces] # optimal cycle plot
for n, trace in enumerate(traces_optimal): # plot optimal cycle
    trace.update(
            showlegend=(n==0),
            legendgroup='opti',
            opacity=0.5,
            name='Optimal Cycle'
        )
fig = go.Figure(data=[trace_dta]+traces_optimal)
fig.update_layout(
        width=400, 
        height=500,
        margin=dict(l=20, r=20, t=20, b=20)
    )
fig.show()
# rounding solves this problem (I think)

Removing 3721/3751 simplicies from the cycle, 30 simplicies left


In [9]:
## Largest Representative 3D Hole (Not Optimized)
# cycle to optimize
i = homology[homology['dimension'] == 2].sort_values('cycle nnz').index[-1] # largest 2d cycle (3d void)

# get the cycle
nonoptimal_faces = homology.loc[i, 'cycle representative']['simplex'].tolist()

# plotting
trace_dta = go.Scatter3d( # data plot
        x=dta[:, 0],
        y=dta[:, 1],
        z=dta[:, 2],
        mode='markers',
        showlegend=True,
        name='Data',
        opacity=0.5
    )
traces_nonoptimal = [oat.plot.triangle__trace3d(triangle=tri, coo=dta) for tri in nonoptimal_faces] # optimal cycle plot
for n, trace in enumerate(traces_nonoptimal): # plot optimal cycle
    trace.update(
            showlegend=(n==0),
            legendgroup='opti',
            opacity=0.5,
            name='Non-Optimal cycle'
        )
fig = go.Figure(data=[trace_dta]+traces_nonoptimal)
fig.update_layout(
        width=400, 
        height=500,
        margin=dict(l=20, r=20, t=20, b=20)
    )
fig.show()

In [10]:
## Largest Representative 3D Hole (Optimized)
# cycle to optimize
i = homology[homology['dimension'] == 2].sort_values('cycle nnz').index[-1] # largest 2d cycle (3d void)

# optimization problem
start = time.time()
optimal = factored.optimize_cycle(
        birth_simplex=homology['birth simplex'][i], 
        problem_type='preserve PH basis'
    )
print(f'Optimization took {time.time() - start} secs') # the notebook doesn't tell me how long it takes and I want to know
optimal_faces = optimal['chain']['optimal cycle']['simplex'].tolist() # bounding box of optimal cycle

# # plotting
# trace_dta = go.Scatter3d( # data plot
#         x=dta[:, 0],
#         y=dta[:, 1],
#         z=dta[:, 2],
#         mode='markers',
#         showlegend=True,
#         name='Data',
#         opacity=0.5
#     )
# traces_optimal = [oat.plot.triangle__trace3d(triangle=tri, coo=dta) for tri in optimal_faces] # optimal cycle plot
# for n, trace in enumerate(traces_optimal): # plot optimal cycle
#     trace.update(
#             showlegend=(n==0),
#             legendgroup='opti',
#             opacity=0.5,
#             name='Optimal cycle'
#         )
# fig = go.Figure(data=[trace_dta]+traces_optimal)
# fig.update_layout(
#         width=400, 
#         height=500,
#         margin=dict(l=20, r=20, t=20, b=20)
#     )
# fig.show()


Finished construcing L1 optimization program.
Constraint matrix has 50878 nonzero entries.
Passing program to solver.

Done solving.
MINILP solution: Solution { direction: Minimize, num_vars: 15034, num_constraints: 16566, objective: 70.8639326137705 }
Optimization took 32.24985074996948 secs


In [11]:
## 2nd Largest Representative 3D Hole (Optimized and rounded)
# use optimal dataframe from above (the optmization problem takes too long, i don't wanna do it twice)
filter = optimal.loc['optimal cycle', 'chain']['coefficient'].astype(float).round() != 0 # mark locations where the cofficicents are close to 0 to be removed
print(f'Removing {sum(1-filter)}/{len(filter)} simplicies from the cycle, {sum(filter)} simplicies left')
optimal_faces = optimal.loc['optimal cycle', 'chain'][filter]['simplex'].tolist()
# maybe want some assert statement to make sure we have a real cycle

# plotting
trace_dta = go.Scatter3d( # data plot
        x=dta[:, 0],
        y=dta[:, 1],
        z=dta[:, 2],
        mode='markers',
        showlegend=True,
        name='Data',
        opacity=0.5
    )
traces_optimal = [oat.plot.triangle__trace3d(triangle=tri, coo=dta) for tri in optimal_faces] # optimal cycle plot
for n, trace in enumerate(traces_optimal): # plot optimal cycle
    trace.update(
            showlegend=(n==0),
            legendgroup='opti',
            opacity=0.5,
            name='Optimal Cycle'
        )
fig = go.Figure(data=[trace_dta]+traces_optimal)
fig.update_layout(
        width=400, 
        height=500,
        margin=dict(l=20, r=20, t=20, b=20)
    )
fig.show()

Removing 2992/3028 simplicies from the cycle, 36 simplicies left
